In [0]:
df = spark.read.format('csv').option("header",True).option("path","/Volumes/workspace/default/department/departments.csv","/Volumes/workspace/default/department/orders.csv","/Volumes/workspace/default/department/customers.csv").load()
display(df)

In [0]:
df = spark.read.format("csv").option("header", True).load([
    "/Volumes/workspace/default/department/departments.csv"
])
display(df)

In [0]:
df1 = spark.read.format("csv").option("header", True).load([
    "/Volumes/workspace/default/department/orders.csv"
])
display(df1)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,rank,dense_rank
from pyspark.sql import *
df_window = Window.partitionBy("product").orderBy("cust_id")
df_rownum = df1.withColumn("row_number",row_number().over(df_window).withcolumn("rank",rank().over(df_window)).withcolumn("dense_rank",dense_rank().over(df_window)))
display(df_rownum)



In [0]:
df_window = Window.partitionBy("product").orderBy("product")

df_rownum = (
    df1
    .withColumn("row_number", row_number().over(df_window))
    .withColumn("rank", rank().over(df_window))
    .withColumn("dense_rank", dense_rank().over(df_window))
)

display(df_rownum)

In [0]:
df4 = df.withColumnRenamed("dept_id","dep_id")
display(df4)

In [0]:
from pyspark.sql.functions import col
df5 = df.sort("location","dept_id")
df6 = df.withColumn("vat", col("dept_id") * 2)
display(df5)
display(df6)

In [0]:
from pyspark.sql.functions import col

df5 = df.sort("location", "dept_id")
df6 = df.withColumn("dept_id_double", col("dept_id") * 2)

display(df5)
display(df6)

In [0]:
from pyspark.sql.functions import col 
df7 = df.filter(col('dept_id').between(10,20))
display(df7)

In [0]:
df9 = spark.read.format("csv").option("header",True).option("path","/Volumes/workspace/default/department/orders.csv").load()
display(df9)
df10 = df9.drop("amount")
display(df10)
df11 = df.union(df10)
display(df11)

In [0]:
from pyspark.sql.functions import col
df12 = df.filter(col("dept_name").endswith("R"))
display(df12)

In [0]:
from pyspark.sql.functions import *
df13 = df.select(max("dept_id").alias("max_dept_id"))
display(df13)


In [0]:
display(df)
df14 = df.groupby(col("dept_name"))

In [0]:
df.createTempView("temp1")
spark.catalog("temp1")

In [0]:
spark.catalog.temp1

In [0]:
%sql
select * from temp1

In [0]:
df15 = df.withColumn("dept_level", when(col("dept_id").cast("int") > 10 , "High").otherwise("low"))
df16 = df15.withColumn("location", upper(col("location")))
display(df16)
display(df15)

In [0]:
from delta.tables import *

DeltaTable.createIfNotExists(spark) \
    .tableName("temp1") \
    .addColumn("dept_id", "STRING") \
    .addColumn("dept_name", "STRING") \
    .addColumn("location", "STRING") \
    .execute()

In [0]:
%sql
select * from temp1;
insert into temp1 values (1,"ECE","Eluru")

In [0]:
%sql
update temp1 set dept_id = 2 where dept_name = "ECE";
select * from temp1;

In [0]:
from delta.tables import *
df =DeltaTable.forName(spark,"temp1")
df.restoreToVersion(2)

In [0]:
%sql
select * from temp1;

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("dept_id", StringType(), True),
    StructField("dept_name", StringType(), True),
    StructField("location", StringType(), True)
])

empty_df = spark.createDataFrame([], schema)

empty_df.write.format("delta").saveAsTable("temp1")

In [0]:
from delta.tables import DeltaTable
df = DeltaTable.forName(spark,"temp1")
df.update("dept_id = 1", {"dept_name" : "MECH"})
display(temp1)

In [0]:
'''You need to use a pyspark.sql.functions.lit() expression for string values in the update dictionary, and display the table using a DataFrame, not the table name.'''
from delta.tables import DeltaTable
from pyspark.sql.functions import lit

delta_table = DeltaTable.forName(spark, "temp1")
delta_table.update(
    condition="dept_id = 1",
    set={"dept_name": lit("MECH")}
)
display(spark.table("temp1"))

In [0]:
'''MERGE INTO employees t
USING new_employees s
ON t.emp_id = s.emp_id
WHEN MATCHED THEN 
  UPDATE SET 
    t.emp_name = s.emp_name,
    t.salary = s.salary
WHEN NOT MATCHED THEN
  INSERT (emp_id, emp_name, salary)
  VALUES (s.emp_id, s.emp_name, s.salary);
'''

In [0]:
target.merge(
    source,
    "t.id = s.id"
).whenMatchedUpdate(
    set = {
        "t.salary": "s.salary",
        "t.location": "s.location"
    }
).execute()


In [0]:
from delta.tables import DeltaTable

DeltaTable.createIfNotExists(spark) \
    .tableName("temp2") \
    .addColumn("id", "int") \
    .addColumn("name", "string") \
    .addColumn("salary", "double") \
    .addColumn("location", "string") \
    .execute()

In [0]:
from delta.tables import DeltaTable

delta_temp2 = DeltaTable.forName(spark, "temp2")
delta_temp1 = spark.table("df")

delta_temp2.alias("t").merge(
    source=delta_temp1.alias("s"),
    condition="t.id = s.dept_id"
).whenMatchedUpdate(
    set={
        "name": "s.dept_name",
        "location": "s.location"
    }
).whenNotMatchedInsert(
    values={
        "id": "s.dept_id",
        "name": "s.dept_name",
        "location": "s.location"
    }
).execute()

In [0]:
df = spark.read.table("temp1")
display(df)

In [0]:

df = spark.read.table("temp3")
display(df)

In [0]:
dbutils.widgets.help()
dbutils.widgets.text("name","prem")
dbutils.widgets.get("name")

In [0]:
dbutils.widgets.text("date", "2024-01-01")
dbutils.widgets.get("date")


In [0]:
dbutils.secrets.
